In [12]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sumn2u/garbage-classification-v2")

print("Path to dataset files:", path)

Path to dataset files: /home/pasha/.cache/kagglehub/datasets/sumn2u/garbage-classification-v2/versions/8


In [13]:
import os
import cv2
import numpy as np
import pandas as pd
import warnings

from skimage.feature import graycomatrix, graycoprops
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from collections import defaultdict

# Suppress warnings for better output
warnings.filterwarnings("ignore", category=UserWarning)

classes = os.listdir(os.path.join(path, "garbage-dataset"))
print(classes)

['trash', 'glass', 'battery', 'clothes', 'metal', 'plastic', 'cardboard', 'paper', 'biological', 'shoes']


## Извлечение фичей

In [14]:
def calculate_yellow_amount(image):
    """Calculate the amount of yellow color in the image."""
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([30, 255, 255])
    yellow_mask = cv2.inRange(hsv_image, lower_yellow, upper_yellow)
    yellow_amount = cv2.countNonZero(yellow_mask)
    return yellow_amount

def calculate_silver_amount(image):
    """Calculate the amount of silver color in the image."""
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_silver = np.array([0, 0, 200])
    upper_silver = np.array([180, 25, 255])
    silver_mask = cv2.inRange(hsv_image, lower_silver, upper_silver)
    silver_amount = cv2.countNonZero(silver_mask)
    return silver_amount

def calculate_parallel_lines(image):
    """Calculate the amount of parallel lines in the image."""
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray_image, 50, 150, apertureSize=3)
    
    # Detect lines using Hough Transform
    lines = cv2.HoughLines(edges, 1, np.pi / 180, threshold=100)
    
    if lines is not None:
        return len(lines)  # Return the number of detected lines
    return 0  # No lines detected

def detect_cylinders(image):
    """Detect cylindrical shapes in the image."""
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(gray_image, (9, 9), 2)

    # Detect circles using Hough Transform
    circles = cv2.HoughCircles(blurred_image, cv2.HOUGH_GRADIENT, dp=1, minDist=30,
                               param1=100, param2=30, minRadius=10, maxRadius=100)

    if circles is not None:
        return circles.shape[1]  # Return the number of detected circles
    return 0  # No circles detected

def calculate_reflection(image):
    """Calculate the amount of bright areas in the image to detect reflections."""
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Threshold to find bright areas
    _, bright_mask = cv2.threshold(gray_image, 200, 255, cv2.THRESH_BINARY)
    reflection_amount = cv2.countNonZero(bright_mask)
    return reflection_amount

def calculate_transparency(image):
    """
    Calculate the amount of transparent areas in the image.
    Transparent areas are identified as regions with low saturation and brightness in the HSV color space.
    """
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # Define thresholds for low saturation and brightness (transparent regions)
    lower_transparent = np.array([0, 0, 0])
    upper_transparent = np.array([180, 50, 50])  # Adjust upper threshold as needed
    transparent_mask = cv2.inRange(hsv_image, lower_transparent, upper_transparent)
    transparency_amount = cv2.countNonZero(transparent_mask)
    return transparency_amount

def calculate_texture_smoothness(image):
    """Calculate the smoothness of the texture in the image."""
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    glcm = graycomatrix(gray_image, [1], [0], 256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    return contrast

def calculate_shininess(image):
    """Calculate the shininess of the image based on bright pixels."""
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    bright_pixels = np.sum(gray_image > 220)  # Count pixels with intensity greater than 220
    total_pixels = image.shape[0] * image.shape[1]
    shininess = bright_pixels / total_pixels  # Proportion of bright pixels
    return shininess

def get_features(image):
    """Get a set of features (yellow amount, silver amount, parallel lines, cylindrical shapes, reflections, transparency, texture smoothness, shininess)."""
    yellow_amount = calculate_yellow_amount(image)
    silver_amount = calculate_silver_amount(image)
    parallel_lines = calculate_parallel_lines(image)
    cylinder_count = detect_cylinders(image)
    reflection_amount = calculate_reflection(image)
    transparency_amount = calculate_transparency(image)
    texture_smoothness = calculate_texture_smoothness(image)
    shininess = calculate_shininess(image)  # Added shininess feature
    return yellow_amount, silver_amount, parallel_lines, cylinder_count, reflection_amount, transparency_amount, texture_smoothness, shininess

In [15]:
# Feature extraction testing
test_im = cv2.imread(path + "/garbage-dataset/cardboard/cardboard_25.jpg")
print(get_features(test_im))

(15834, 284114, 140, 15, 303454, 0, np.float64(150.46217028380636), np.float64(0.8047277777777778))


## Обучение классификатора

In [16]:
# Example feature extraction
test_im = cv2.imread(path + "/garbage-dataset/cardboard/cardboard_25.jpg")
print(get_features(test_im))

# Initialize lists to hold features and labels
features = []
labels = []
test_images = []  # List to hold paths of test images

# Loop through each class and extract features from images
for class_name in classes:
    class_path = os.path.join(path + "/garbage-dataset/", class_name)
    image_names = os.listdir(class_path)

    # Split the images into training and testing sets (80% train, 20% test)
    train_images, test_images_class = train_test_split(image_names, test_size=0.2, random_state=42)

    # Process training images
    for image_name in train_images:
        image_path = os.path.join(class_path, image_name)
        image = cv2.imread(image_path)

        if image is not None:
            features_list = get_features(image)
            features.append(features_list)
            labels.append(class_name)

    # Collect test images for manual testing
    for image_name in test_images_class:
        test_images.append(os.path.join(class_path, image_name))

# Convert features and labels to a DataFrame
df = pd.DataFrame(features, columns=['yellow_amount', 'silver_amount', 'parallel_lines', 'cylinder_count', 'reflection_amount', 'transparency_amount', 'texture_smoothness', 'shininess'])
df['label'] = labels

# Split the dataset into training and testing sets
X = df[['yellow_amount', 'silver_amount', 'parallel_lines', 'cylinder_count', 'reflection_amount', 'transparency_amount', 'texture_smoothness', 'shininess']]
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Classifier
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Function to classify a new image
def classify_image(image_path):
    image = cv2.imread(image_path)
    if image is not None:
        features_list = get_features(image)
        feature = np.array([features_list])  # Convert to 2D array for prediction
        prediction = classifier.predict(feature)
        return prediction[0]
    else:
        return None

(15834, 284114, 140, 15, 303454, 0, np.float64(150.46217028380636), np.float64(0.8047277777777778))
Accuracy: 0.5825426944971537
              precision    recall  f1-score   support

     battery       0.67      0.59      0.63       145
  biological       0.52      0.63      0.57       138
   cardboard       0.66      0.56      0.61       311
     clothes       0.66      0.93      0.78       886
       glass       0.45      0.51      0.48       455
       metal       0.33      0.10      0.15       163
       paper       0.57      0.39      0.46       263
     plastic       0.53      0.33      0.41       327
       shoes       0.50      0.44      0.47       317
       trash       0.55      0.43      0.48       157

    accuracy                           0.58      3162
   macro avg       0.54      0.49      0.50      3162
weighted avg       0.57      0.58      0.56      3162



In [17]:
# Classifier testing (runs classifier for a specific image)
result = classify_image(path + '/garbage-dataset/metal/metal_2370.jpg') 
print("Predicted class:", result)

Predicted class: glass


In [18]:
# Classifier testing (runs classifier for the remaining images)
def test_classify(test_images, target_class_name):
    predictions = defaultdict(int)
    for test_image in test_images:
        if target_class_name in test_image:  
            predicted_class = classify_image(test_image)  
            predictions[predicted_class] += 1 
    return dict(predictions)  

for class_name in classes:
    results = test_classify(test_images, class_name)
    print("Predicted class counts for test images containing " + class_name + ':')
    for class_name, count in results.items():
        print(f"Class: {class_name}, Count: {count}")

Predicted class counts for test images containing trash:
Class: metal, Count: 3
Class: trash, Count: 84
Class: clothes, Count: 21
Class: plastic, Count: 6
Class: shoes, Count: 12
Class: paper, Count: 6
Class: glass, Count: 38
Class: cardboard, Count: 13
Class: battery, Count: 2
Class: biological, Count: 5
Predicted class counts for test images containing glass:
Class: glass, Count: 303
Class: trash, Count: 19
Class: metal, Count: 10
Class: plastic, Count: 46
Class: shoes, Count: 64
Class: clothes, Count: 96
Class: battery, Count: 18
Class: cardboard, Count: 13
Class: biological, Count: 26
Class: paper, Count: 18
Predicted class counts for test images containing battery:
Class: shoes, Count: 28
Class: glass, Count: 35
Class: battery, Count: 109
Class: biological, Count: 6
Class: paper, Count: 1
Class: clothes, Count: 6
Class: plastic, Count: 1
Class: cardboard, Count: 2
Class: metal, Count: 1
Predicted class counts for test images containing clothes:
Class: clothes, Count: 972
Class: pa

## Проверка гипотез